In [3]:
import xarray as xr

path = r"E:\Subham_2025\Oxygen\woa23_all_o00_01.nc"

# Disable automatic time decoding
ds = xr.open_dataset(path, engine="netcdf4", decode_times=False)

print("\n✅ Dataset opened successfully!")
print(ds)



✅ Dataset opened successfully!
<xarray.Dataset> Size: 291MB
Dimensions:             (lat: 180, nbounds: 2, lon: 360, depth: 102, time: 1)
Coordinates:
  * lat                 (lat) float32 720B -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * lon                 (lon) float32 1kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * depth               (depth) float32 408B 0.0 5.0 10.0 ... 5.4e+03 5.5e+03
  * time                (time) float32 4B 3.894e+03
Dimensions without coordinates: nbounds
Data variables: (12/14)
    crs                 int32 4B ...
    lat_bnds            (lat, nbounds) float32 1kB ...
    lon_bnds            (lon, nbounds) float32 3kB ...
    depth_bnds          (depth, nbounds) float32 816B ...
    climatology_bounds  (time, nbounds) float32 8B ...
    o_an                (time, depth, lat, lon) float32 26MB ...
    ...                  ...
    o_sd                (time, depth, lat, lon) float32 26MB ...
    o_se                (time, depth, lat, lon) float32 26MB ...
    o_oa   

In [55]:
import xarray as xr
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# ===============================
# 1️⃣ Load dataset ignoring time
# ===============================
path = r"E:\Subham_2025\Oxygen\woa23_all_o00_01.nc"
ds = xr.open_dataset(path, decode_times=False)

lon = ds['lon'].values
lat = ds['lat'].values
depths = ds['depth'].values
oxygen = ds['o_an'].isel(time=0)

lon2d, lat2d = np.meshgrid(np.deg2rad(lon), np.deg2rad(lat))
x = np.cos(lat2d) * np.cos(lon2d)
y = np.cos(lat2d) * np.sin(lon2d)
z = np.sin(lat2d)

# ===============================
# 2️⃣ Create land overlay
# ===============================
land_mask = np.where(np.isnan(oxygen.isel(depth=0).values), 1, 0)

# ===============================
# 3️⃣ Create frames for each depth
# ===============================
frames = []
for i, d in enumerate(depths):
    data = np.where(np.isnan(oxygen.isel(depth=i).values), 0, oxygen.isel(depth=i).values)
    frames.append(go.Frame(
        data=[
            go.Surface(
                x=x, y=y, z=z,
                surfacecolor=data,
                colorscale='icefire',
                cmin=np.nanmin(oxygen.values),
                cmax=np.nanmax(oxygen.values),
                colorbar=dict(title="Oxygen (µmol/kg)"),
                hovertemplate="Lat: %{y:.2f}<br>Lon: %{x:.2f}<br>Oxygen: %{surfacecolor:.2f} µmol/kg<extra></extra>"
            ),
            go.Surface(
                x=x, y=y, z=z + 0.01,
                surfacecolor=land_mask,
                colorscale=[[0, 'rgba(0,0,0,0)'], [1, 'green']],
                showscale=False,
                opacity=0.6
            )
        ],
        name=f"{d:.1f}",
        layout=go.Layout(
            title=f"🌍 Dissolved Oxygen at Depth = {d:.1f} m"
        )
    ))

# ===============================
# 4️⃣ Initial surface
# ===============================
data0 = np.where(np.isnan(oxygen.isel(depth=0).values), 0, oxygen.isel(depth=0).values)

# ===============================
# 5️⃣ Globe outline
# ===============================
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 50)
xs = np.outer(np.cos(u), np.sin(v))
ys = np.outer(np.sin(u), np.sin(v))
zs = np.outer(np.ones_like(u), np.cos(v))

# ===============================
# 6️⃣ Create figure
# ===============================
fig = go.Figure(
    data=[
        go.Surface(
            x=x, y=y, z=z,
            surfacecolor=data0,
            colorscale='icefire',
            cmin=np.nanmin(oxygen.values),
            cmax=np.nanmax(oxygen.values),
            colorbar=dict(title="Oxygen (µmol/kg)"),
            hovertemplate="Lat: %{y:.2f}<br>Lon: %{x:.2f}<br>Oxygen: %{surfacecolor:.2f} µmol/kg<extra></extra>"
        ),
        go.Surface(
            x=x, y=y, z=z + 0.01,
            surfacecolor=land_mask,
            colorscale=[[0, 'rgba(0,0,0,0)'], [1, 'green']],
            showscale=False,
            opacity=0.6
        ),
        go.Surface(
            x=xs, y=ys, z=zs,
            colorscale=[[0, 'lightblue'], [1, 'lightblue']],
            opacity=0.05,
            showscale=False
        )
    ],
    layout=go.Layout(
        title=f"🌍 Dissolved Oxygen at Depth = {depths[0]:.1f} m",
        scene=dict(
            xaxis=dict(visible=False, showgrid=True, gridcolor='white'),
            yaxis=dict(visible=False, showgrid=True, gridcolor='white'),
            zaxis=dict(visible=False, showgrid=False),
            aspectmode='data',
            bgcolor='rgba(0,0,0,0)'
        ),
        paper_bgcolor='black',
        font=dict(color='white'),
        margin=dict(l=0, r=0, t=50, b=0),
        sliders=[dict(
            active=0,
            currentvalue={"prefix": "Depth (m): ", "font": {"color": "white"}},
            pad={"t": 50},
            steps=[dict(
                method="animate",
                args=[[f"{d:.1f}"], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}, "transition": {"duration": 0}}],
                label=f"{d:.1f}"
            ) for d in depths]
        )],
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            x=0.0,
            y=0.0,
            xanchor="left",
            yanchor="bottom",
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[None, {"frame": {"duration": 200, "redraw": True},
                                  "fromcurrent": True, "transition": {"duration": 0}}]),
                dict(label="Pause",
                     method="animate",
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ]
        )],
        images=[dict(
            source=r"E:\Subham_2025\Oxygen\ChatGPT Image Nov 7, 2025, 12_11_25 PM.png",
            xref="paper", yref="paper",
            x=0, y=1,
            sizex=1, sizey=1,
            xanchor="left",
            yanchor="top",
            layer="below",
            sizing="stretch",
            opacity=1
        )]
    ),
    frames=frames
)

# ===============================
# 7️⃣ Save interactive HTML
# ===============================
pio.write_html(fig, file="oxygen_surface_interactive_globe.html", auto_open=True)